In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from kerastuner.tuners import RandomSearch
%matplotlib inline

In [2]:
input_shape = (194, 259, 3)
num_classes = 2

data_dir = "/media/liam/F8EEAC88EEAC412C/Users/liam_/dev/tensorflow/data"
model_dir = "/run/user/1000/gvfs/smb-share:server=raspberrypi.local,share=hasty"

In [3]:
class DataGenerator:
    
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = sorted(os.listdir(f"{data_dir}/x"))
        self.files_i = 0
        self.file = self.files[self.files_i]
        self.data_i = 0
        self.x = np.load(f"{self.data_dir}/x/{self.file}", allow_pickle=True)
        self.y = np.load(f"{self.data_dir}/y/{self.file}", allow_pickle=True)

    def grab_next(self):

        if self.data_i >= len(self.x):
            self.files_i = (self.files_i + 1) % len(self.files)
            self.file = self.files[self.files_i]
            self.data_i = 0
            self.x = np.load(f"{self.data_dir}/x/{self.file}", allow_pickle=True)
            self.y = np.load(f"{self.data_dir}/y/{self.file}", allow_pickle=True)
            self.len_data = len(self.x)

        self.data_i += 1

        return self.x[self.data_i], self.y[self.data_i]
        
    def take(self, n):
        xy = np.array([data_generator.grab_next() for _ in range(n)])
        np.random.shuffle(xy)
        return np.array([i for i in xy[:, 0]]), xy[:, 1].astype(np.uint8)

In [4]:
def plot_model(history):
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    # plt.ylim([0.5, 1])
    plt.legend(loc='lower right')
    plt.show()
    
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label = 'val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    # plt.ylim([0.5, 1])
    plt.legend(loc='lower right')
    plt.show()

In [5]:
def build_model(hp):
        model = keras.Sequential()
        model.add(keras.layers.AveragePooling2D(6, 3, input_shape=input_shape))
        
        for i in range(hp.Int("Conv Layers", min_value=1, max_value=3)):
            model.add(keras.layers.Conv2D(hp.Choice(f"layer_{i}_filters", [16, 32, 64]), hp.Choice(f"Conv_size", [3, 5]), activation="relu"))
        
        model.add(keras.layers.MaxPool2D(2, 2))
        model.add(keras.layers.Dropout(hp.Choice(f"Dropout rate", [0.1, 0.2, 0.3])))
        model.add(keras.layers.Flatten())
        
        model.add(keras.layers.Dense(hp.Choice("Dense layer", [64, 128, 256, 512]), activation="relu"))
        model.add(keras.layers.Dense(num_classes, activation="softmax"))

        model.compile(optimizer="adam",
                     loss=keras.losses.SparseCategoricalCrossentropy(),
                     metrics=["accuracy"])
        return model

In [ ]:
data_generator = DataGenerator(data_dir)

with tf.device('/GPU:0'):
    
    data_size = 500
    test_percent = 0.1
    test_size = int(data_size * test_percent)
    train_size = data_size - test_size

    x_train, y_train = data_generator.take(train_size)
    x_test, y_test = data_generator.take(test_size)
    
    with tf.device('/GPU:0'):  
        tuner = RandomSearch(
            build_model,
            objective="val_accuracy",
            max_trials=32,
            executions_per_trial=2,
            directory=f"{model_dir}/charts-cnn-model",
            project_name="charts-cnn-model"
        )
        
        tuner.search(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=20)


INFO:tensorflow:Reloading Oracle from existing project /run/user/1000/gvfs/smb-share:server=raspberrypi.local,share=hasty/charts-cnn-model/charts-cnn-model/oracle.json
INFO:tensorflow:Reloading Tuner from /run/user/1000/gvfs/smb-share:server=raspberrypi.local,share=hasty/charts-cnn-model/charts-cnn-model/tuner0.json

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
Conv Layers       |2                 |1                 
Dense layer       |1024              |256               
layer_0_filters   |16                |32                
layer_1_filters   |16                |32                
layer_2_filters   |32                |None              

Epoch 1/3
23/23 [==============================] - ETA: 0s - loss: 0.8264 - accuracy: 0.5333

In [ ]:
best_model = tuner.get_best_models()[0]
best_model.summary()

In [ ]:
tuner.results_summary()

In [ ]:
best_model.save("./my_model")